In [157]:
import pandas as pd
import numpy as np
import vaex
import pickle

In [165]:
#read biblio_acci

acci = pd.read_csv('../data/biblio_acci.csv')

acci

,unir,accidente
0,2013-01-01 00:00:00 4579,1
1,2013-01-01 00:00:00 4608,1
2,2013-01-01 02:30:00 3974,1
3,2013-01-01 04:00:00 3619,1
4,2013-01-01 07:30:00 9949,1
...,...,...
109476,2022-12-31 21:00:00 6445,1
109477,2022-12-31 21:00:00 6757,1
109478,2022-12-31 22:00:00 4182,1
109479,2022-12-31 23:00:00 4254,1


In [4]:
traf = vaex.open('../../TRAFICO/traf_hist_unir.hdf5')

traf.info()

In [110]:
traf_6757 = traf[traf.idelem == '3885']

traf_6757 = traf_6757.to_pandas_df()

traf_6757_acci = pd.merge(traf_6757, acci, on='unir', how='left')

traf_6757_acci['accidente'] = traf_6757_acci['accidente'].fillna(0)

In [111]:
traf_6757_acci.accidente.value_counts()

0.00    153089
1.00        29
Name: accidente, dtype: int64

In [166]:
id = '6757'

#filtrar por idelem

traf_6757 = traf[traf.idelem == id]

print('ya he filtrado por idelem')

#generar tabla para modelo

traf_6757 = traf_6757.to_pandas_df()

traf_6757_acci = pd.merge(traf_6757, acci, on='unir', how='left')

traf_6757_acci['accidente'] = traf_6757_acci['accidente'].fillna(0)

traf_6757_acci.drop(['unir', 'idelem'], axis=1, inplace=True)

traf_6757_acci['fecha'] = traf_6757_acci['fecha_full'].str.split(' ').str[0]

traf_6757_acci['hora'] = traf_6757_acci['fecha_full'].str.split(' ').str[1]

traf_6757_acci.drop('fecha_full', axis=1, inplace=True)

traf_6757_acci['prec'] = pd.to_numeric(traf_6757_acci['prec'].str.replace(',', '.'), errors='coerce')

traf_6757_acci['velmedia'] = pd.to_numeric(traf_6757_acci['velmedia'].str.replace(',', '.'), errors='coerce')

traf_6757_acci.fillna(0, inplace=True)

traf_6757_acci['mes'] = traf_6757_acci['fecha'].str.split('-').str[1]

traf_6757_acci['fecha'] = pd.to_datetime(traf_6757_acci['fecha'])

traf_6757_acci['dia_semana'] = traf_6757_acci['fecha'].dt.dayofweek

traf_6757_acci.drop('fecha', axis=1, inplace=True)

traf_6757_acci['dia_semana'] = traf_6757_acci['dia_semana'].astype('object')

#get dummies

print('genero dummies')

traf_6757_acci_dum = pd.get_dummies(traf_6757_acci, columns=['mes', 'dia_semana', 'hora'])

#Xtest, ytest

from sklearn.model_selection import train_test_split

X = traf_6757_acci_dum.drop('accidente', axis=1)

y = traf_6757_acci_dum['accidente']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#scale

print('train test')

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

#model

print('modelo')

model = LogisticRegression()
model.fit(X_train, y_train)

print('pickle')

filename = 'modelo_' + id + '.sav'
pickle.dump(model, open('../modelos/' + filename, 'wb'))

ya he filtrado por idelem


In [164]:
id = '6757'

filename = 'modelo_' + id + '.sav'
pickle.dump(model, open('../modelos/' + filename, 'wb'))

In [160]:
filename = 'modelo_prueba.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [29]:
traf_6757_acci.accidente.value_counts()

0.0    141365
1.0        38
2.0         3
Name: accidente, dtype: int64

In [47]:
traf_6757_acci.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141406 entries, 0 to 141405
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   intensidad  141406 non-null  float64
 1   ocupacion   141406 non-null  float64
 2   carga       141406 non-null  float64
 3   prec        141406 non-null  float64
 4   velmedia    141406 non-null  float64
 5   accidente   141406 non-null  float64
 6   hora        141406 non-null  object 
 7   mes         141406 non-null  object 
 8   dia_semana  141406 non-null  object 
dtypes: float64(6), object(3)
memory usage: 14.8+ MB


In [44]:
traf_6757_acci

,intensidad,ocupacion,carga,prec,velmedia,accidente,hora,mes,dia_semana
0,1044.0,4.0,24.0,0.7,3.3,0.0,20:30:00,01,6
1,1116.0,4.0,25.0,0.7,3.3,0.0,20:00:00,01,6
2,768.0,3.0,18.0,0.7,3.3,0.0,21:30:00,01,6
3,768.0,3.0,17.0,0.7,3.3,0.0,21:00:00,01,6
4,648.0,3.0,15.0,0.7,3.3,0.0,22:00:00,01,6
...,...,...,...,...,...,...,...,...,...
141401,252.0,6.0,20.0,0.0,0.8,0.0,21:30:00,12,5
141402,439.0,1.0,29.0,0.0,0.8,0.0,22:00:00,12,5
141403,268.0,1.0,17.0,0.0,0.8,0.0,22:30:00,12,5
141404,205.0,0.0,12.0,0.0,0.8,0.0,23:00:00,12,5


In [49]:
traf_6757_acci_dum

,intensidad,ocupacion,carga,prec,velmedia,accidente,mes_01,mes_02,mes_03,mes_04,...,hora_19:00:00,hora_19:30:00,hora_20:00:00,hora_20:30:00,hora_21:00:00,hora_21:30:00,hora_22:00:00,hora_22:30:00,hora_23:00:00,hora_23:30:00
0,1044.0,4.0,24.0,0.7,3.3,0.0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1116.0,4.0,25.0,0.7,3.3,0.0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,768.0,3.0,18.0,0.7,3.3,0.0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,768.0,3.0,17.0,0.7,3.3,0.0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,648.0,3.0,15.0,0.7,3.3,0.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141401,252.0,6.0,20.0,0.0,0.8,0.0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
141402,439.0,1.0,29.0,0.0,0.8,0.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
141403,268.0,1.0,17.0,0.0,0.8,0.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
141404,205.0,0.0,12.0,0.0,0.8,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


## Entrenar modelos

In [115]:
#split train and test

from sklearn.model_selection import train_test_split

X = traf_6757_acci_dum.drop('accidente', axis=1)

y = traf_6757_acci_dum['accidente']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#scale

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [84]:
# fit models with class weights and get the f1 score

from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier

from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import AdaBoostClassifier

from sklearn.ensemble import BaggingClassifier

from sklearn.ensemble import ExtraTreesClassifier

from sklearn.neural_network import MLPClassifier

from sklearn.metrics import confusion_matrix

In [72]:
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights

class_weights = compute_class_weight(class_weight = "balanced", classes = np.unique(y_train), y = y_train)


In [151]:
# Define and fit the model with class weights
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Compute F1 score and print the result
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 score for Logistic Regression: {f1:.2f}")

scores = cross_val_score(clf, X, y, cv=5, scoring='f1_weighted')

# Print the cross-validation scores
print(f"Cross-validation F1 scores: {scores}")
print(f"Mean F1 score: {scores.mean():.2f}")
print(f"Standard deviation: {scores.std():.2f}")

F1 score for Logistic Regression: 1.00
Cross-validation F1 scores: [0.99970612 0.99970612 0.99970612 0.99975509 0.99970611]
Mean F1 score: 1.00
Standard deviation: 0.00


In [87]:
clf = RandomForestClassifier(n_estimators=100, max_depth=5, class_weight='balanced')
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Compute F1 score and print the result
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 score for Random Forest Classifier: {f1:.2f}")

scores = cross_val_score(clf, X, y, cv=5, scoring='f1_weighted')

# Print the cross-validation scores
print(f"Cross-validation F1 scores: {scores}")
print(f"Mean F1 score: {scores.mean():.2f}")
print(f"Standard deviation: {scores.std():.2f}")

F1 score for Random Forest Classifier: 0.96
Cross-validation F1 scores: [0.99923805 0.99621059 0.95092374 0.94750466 0.74700885]
Mean F1 score: 0.93
Standard deviation: 0.09


In [133]:

# Define and fit the model with class weights
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Compute F1 score and print the result
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 score for Decision Tree Classifier: {f1:.2f}")

scores = cross_val_score(clf, X, y, cv=5, scoring='f1_weighted')

# Print the cross-validation scores
print(f"Cross-validation F1 scores: {scores}")
print(f"Mean F1 score: {scores.mean():.2f}")
print(f"Standard deviation: {scores.std():.2f}")

F1 score for Decision Tree Classifier: 1.00
Cross-validation F1 scores: [0.99918338 0.99960815 0.99962448 0.99959179 0.99957548]
Mean F1 score: 1.00
Standard deviation: 0.00


In [152]:
ypred = clf.predict_proba(X_test)

In [154]:
#value counts of the predictions



Float64Index([ 0.6306733116242675,  0.6069207665016638,  0.5977723645288638,
               0.5524398332750278,  0.5305866119421804,   0.522769665516912,
              0.49428096587235576, 0.47082177236798667,  0.4626793379022707,
              0.46263620532779354,  0.4446165006499212, 0.43260695216967804,
               0.3985760285256784, 0.39828076142416136, 0.39615264516216747,
               0.3896680348969015,  0.3877680316641949, 0.37400855698066715,
              0.37118945780370394, 0.36989834749255085,  0.3633620713127149,
              0.36264957252007074,  0.3620322061506968,  0.3564901739793292,
              0.35612633374477426,  0.3543392421487287, 0.35184387317784305,
               0.3478326672040331,  0.3465386997443449,  0.3453607406813089,
              0.33813573442248807,   0.334477344214937, 0.33330705937794924,
               0.3320669073925838, 0.33009139074231597,  0.3299087891004454,
               0.3238014703061728, 0.32367995660602655, 0.32293733203956343,

In [116]:
scores = cross_val_score(clf, X, y, cv=5, scoring='f1_weighted')

# Print the cross-validation scores
print(f"Cross-validation F1 scores: {scores}")
print(f"Mean F1 score: {scores.mean():.2f}")
print(f"Standard deviation: {scores.std():.2f}")

Cross-validation F1 scores: [0.99965714 0.9996898  0.99949383 0.99960812 0.9996408 ]
Mean F1 score: 1.00
Standard deviation: 0.00


In [155]:
clf

LogisticRegression()

In [73]:
from lazypredict.Supervised import LazyRegressor

reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)

In [ ]:
models, predictions = reg.fit(X_train, y_train, sample_weight=class_weights)

In [56]:
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
NuSVR,-0.00,0.00,0.02,654.52
OrthogonalMatchingPursuitCV,-0.00,0.00,0.02,1.36
LassoCV,-0.00,0.00,0.02,2.31
ElasticNetCV,-0.00,0.00,0.02,1.85
LassoLarsIC,-0.00,0.00,0.02,0.80
Lars,-0.00,0.00,0.02,0.37
OrthogonalMatchingPursuit,-0.00,0.00,0.02,0.36
LarsCV,-0.00,0.00,0.02,1.51
LassoLarsCV,-0.00,0.00,0.02,1.81


In [57]:
predictions

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
NuSVR,-0.00,0.00,0.02,654.52
OrthogonalMatchingPursuitCV,-0.00,0.00,0.02,1.36
LassoCV,-0.00,0.00,0.02,2.31
ElasticNetCV,-0.00,0.00,0.02,1.85
LassoLarsIC,-0.00,0.00,0.02,0.80
Lars,-0.00,0.00,0.02,0.37
OrthogonalMatchingPursuit,-0.00,0.00,0.02,0.36
LarsCV,-0.00,0.00,0.02,1.51
LassoLarsCV,-0.00,0.00,0.02,1.81
